In [1]:
import os
import logging
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, losses, optimizers, metrics
from  tqdm import *

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
# tf.keras.backend.set_floatx('float32')
import os 

In [2]:
# 全局参数
NUM_USER = 100
np.random.seed(12)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(filename)s[line:%(lineno)d] - %(levelname)s: %(message)s')
import warnings
warnings.filterwarnings("ignore")

In [3]:
# 可调参数
loss_fn = losses.SparseCategoricalCrossentropy()
metrics_list = [tf.keras.metrics.Accuracy()]
# optimizer = optimizers.SGD(learning_rate=1e-2, )
# optimizer = optimizers.SGD(learning_rate=1e-2, momentum=0.8)
optimizer_class = optimizers.SGD
E = 3
batch_size = 20
hidden_unit = 100
beta = 0.99

In [4]:
import pickle

# 加载数据
with open('../data/mnist/train_array.pkl', 'rb') as f:
    train_data = pickle.load(f)
with open('../data/mnist/test_array.pkl', 'rb') as f:
    test_data = pickle.load(f)

In [5]:
def normalization(x):
    return x / np.sum(x)

class MnistModel(tf.Module):
    def __init__(self, hidden_unit):
        """

        :param
            hidden_unit: positive integer, dimensionality of the hidden layer.
        """
        super(MnistModel, self).__init__()
        self.hidden_unit = hidden_unit
        self.fc_1 = layers.Dense(hidden_unit)
        self.fc_2 = layers.Dense(10, activation='softmax')
        
    def __call__(self, x):
        x = layers.ReLU()(self.fc_1(x))
        output = self.fc_2(x)
        return output

## 客户端模型

In [26]:
class Client():
    def __init__(self, id, hidden_unit, train_data={'x':[], 'y':[]}, 
                 test_data={'x':[], 'y':[]}, **kwargs):
        self.id = id
        self.train_data = {'x':np.array(train_data['x']), 
                           'y':np.array(train_data['y'])}
        self.test_data = {'x':np.array(test_data['x']), 
                           'y':np.array(test_data['y'])}
        self.train_samples = len(train_data['y'])
        self.test_samples = len(test_data['y'])
        self.local_model = MnistModel(hidden_unit)
        self.global_model = MnistModel(hidden_unit)
        self.index = np.arange(self.train_samples)
        for key, value in kwargs.items():
            setattr(self, key, value)
        
        assert hasattr(self, 'E')
        assert hasattr(self, 'optimizer')  # TODO:这里改成opt_class
        assert hasattr(self, 'lr')  # 
        assert hasattr(self, 'loss_fn')
        assert hasattr(self, 'metrics')
        self.optimizer = kwargs['optimizer'](self.lr)
        self.init_model()
        self.alpha = tf.Variable(tf.random.uniform([1, 1], maxval=1), dtype=tf.float32)
        self.mix_model = MnistModel(hidden_unit)
        self.mixture_model()
    
    def init_model(self):
        init_feature = tf.cast(self.test_data["x"][:5], dtype=tf.float64)
        _ = self.local_model(init_feature)
        _ = self.global_model(init_feature)
    
    def mixture_model(self):
        for i in range(len(self.mix_model.trainable_variables)):
            mixture_value = self.alpha * self.local_model.trainable_variables[i] + (1-self.alpha) * self.global_model.trainable_variables[i]
            tmp_shape = self.mix_model.trainable_variables[i].shape
            self.mix_model.trainable_variables[i].assign(tf.reshape(mixture_value, tmp_shape))
    
    def alpha_grad(self, grads):
        cum_ = 0.0
        for v, w, grad in zip(self.local_model.trainable_variables,
                              self.global_model.trainable_variables,
                              grads):
            cum_ += tf.reshape(v-w, (1, -1)) @ tf.reshape(grad, (-1, 1))
        return self.lr * cum_
            
    
    def forward(self, communication_round=None):
        np.random.shuffle(self.index)
        self.select_index = self.index[:self.batch_size * self.E]
        train_set = tf.data.Dataset.from_tensor_slices((self.train_data["x"][self.select_index],
                                                        self.train_data["y"][self.select_index])).batch(self.batch_size)
        local_round = 0
        for feature, label in train_set:
            label = tf.cast(label, dtype=tf.int64)
            with tf.GradientTape() as tape:
                predict = self.global_model(feature)
                loss_g = self.loss_fn(label, predict)
            grads_g = tape.gradient(loss_g, self.global_model.trainable_variables)
            self.optimizer.apply_gradients(zip(grads_g, self.global_model.trainable_variables))
            
            with tf.GradientTape() as tape:
                predict = self.mix_model(feature)
                loss_m = self.loss_fn(label, predict)
            grads_m = tape.gradient(loss_m, self.mix_model.trainable_variables)
            for (i, variable) in enumerate(self.local_model.trainable_variables):
                variable.assign_sub(self.lr * grads_m[i])
            # self.optimizer.apply_gradients(zip(grads_m, self.local_model.trainable_variables))
            self.alpha.assign_sub(self.alpha_grad(grads_m))
            self.mixture_model()
            local_round += 1 
        return (self.global_model.trainable_variables)
    
    def update_variables(self, new_variables):
        """
        :param
            new_variables: tuple, each element is ndarray
        :result: None, only copy server model
        """
        assert len(self.global_model.trainable_variables) == len(new_variables)
        for i in range(len(new_variables)):
            self.global_model.trainable_variables[i].assign(new_variables[i])

    
    def test(self):
        test_set = tf.data.Dataset.from_tensor_slices((self.test_data["x"], self.test_data["y"])).batch(self.test_samples)
        train_set = tf.data.Dataset.from_tensor_slices((self.train_data["x"], self.train_data["y"])).batch(self.train_samples)
        for feature, label in test_set:
            label = tf.cast(label, dtype=tf.int64)
            output = self.mix_model(feature)
            loss = self.loss_fn(label, output).numpy()
        
        # logit = tf.nn.softmax(output, axis=-1).numpy()
        prediction = tf.argmax(output, axis=-1).numpy()

        metric_result = []
        for metric in self.metrics:
            metric.reset_states()
            _ = metric.update_state(self.test_data['y'], prediction)
            metric_result.append(metric.result().numpy())
        
        for feature, label in train_set:
            label = tf.cast(label, dtype=tf.int64)
            output = self.mix_model(feature)
            train_loss = self.loss_fn(label, output).numpy()
        # group_idx = tf.argmax(self.model.c, axis=1).numpy()[0]
        return (loss, metric_result, train_loss)

## 服务端模型

In [27]:
class Server():
    def __init__(self, hidden_unit, train_data, test_data, E, optimizer, loss_fn, metrics, batch_size, epoches, lr, filename='../result/mnist_apfl.txt'):
        self.hidden_unit = hidden_unit
        self.train_data = train_data
        self.test_data = test_data
        self.E = E
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        self.metrics = metrics
        self.batch_size = batch_size
        self.epoches = epoches
        self.model = MnistModel(hidden_unit)

        self.init_model()
        self.lastest_model = self.get_parameters()
        self.clients = self.setup_clients(lr)
        self.file = filename

    def init_model(self):
        init_feature = tf.cast(self.test_data[0]['x'][:5], dtype=tf.float64)
        _ = self.model(init_feature)
    
    def get_parameters(self):
        parameter = []
        for variable in self.model.trainable_variables:
            parameter.append(variable.numpy())
        return parameter
    
    def setup_clients(self, lr):
        client_list =[]
        for i in range(NUM_USER):
            client = Client(i, self.hidden_unit, train_data=self.train_data[i], test_data=self.test_data[i],
                                     E=self.E, optimizer=self.optimizer, loss_fn=self.loss_fn, metrics=self.metrics, 
                                     batch_size=self.batch_size, lr=lr)
            client_list.append(client)
            
        return client_list
    
    def broadcast(self):
        for client in self.clients:
            client.update_variables(self.lastest_model)
    
    def train(self):
        for epoch in trange(self.epoches):
            self.broadcast()  # broadcast to all clients
            client_solution = [client.forward(epoch) for client in self.clients]
            self.lastest_model = self.aggragate(client_solution)

            round_loss, round_metrics, train_loss = self.test()
            with open(self.file, 'a+') as f:
                f.write('At round {}, test loss is: {:.4f}, metrics result is {}, train loss is {}'.format(epoch, round_loss, round_metrics, train_loss))
                f.write('\n')
            logging.info('At round {}, test loss is: {:.4f}, metrics result is {}, train loss is {}'.format(epoch, round_loss, round_metrics, train_loss))

            
    def aggragate(self, client_solution):
        concate_V = [np.zeros_like(x) for x in self.lastest_model]
        for i, solution in enumerate(client_solution):
            client_variables = solution
            concate_V = [concate_V[j]+client_variables[j].numpy() for j in range(len(concate_V))]
        V = [element / NUM_USER for element in concate_V]
        return V

    def test(self):
        loss = []
        metrics_list = [[] * len(self.metrics)]
        train_loss = []
        for idx, client in enumerate(self.clients):
            client_loss, client_metrics, client_trainloss = client.test()
            loss.append(client_loss)
            train_loss.append(client_trainloss)
            for i in range(len(metrics_list)):
                metrics_list[i].append(client_metrics[i])
        return np.mean(loss), [np.mean(metrics_list[i]) for i in range(len(metrics_list))], np.mean(train_loss)

In [28]:
np.random.seed(12)
tf.random.set_seed(123)

In [29]:
# server = Server(hidden_unit=hidden_unit, train_data=train_data['user_data'], test_data=test_data['user_data'], E=E, optimizer=optimizer,
# loss_fn=loss_fn, metrics=metrics_list, batch_size=64, epoches=100, lr=1e-2)
server = Server(hidden_unit=hidden_unit, train_data=train_data['user_data'], test_data=test_data['user_data'], E=E, optimizer=optimizer_class,
loss_fn=loss_fn, metrics=metrics_list, batch_size=batch_size, epoches=200, lr=1e-2)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



2021-02-23 00:35:18,195 - base_layer.py[line:2360] - WARNING: Layer dense_2408 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because its dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



2021-02-23 00:35:18,219 - base_layer.py[line:2360] - WARNING: Layer dense_2410 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because its dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



...

2021-02-23 00:35:21,335 - base_layer.py[line:2360] - WARNING: Layer dense_3000 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because its dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



2021-02-23 00:35:21,359 - base_layer.py[line:2360] - WARNING: Layer dense_3004 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because its dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



2021-02-23 00:35:21,365 - base_layer.py[line:2360] - WARNING: Layer dense_3006 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because its dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [30]:
server.train()

  0%|                                                                                          | 0/200 [00:00<?, ?it/s]


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



2021-02-23 00:35:24,934 - base_layer.py[line:2360] - WARNING: Layer dense_2414 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because its dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



2021-02-23 00:35:25,009 - base_layer.py[line:2360] - WARNING: Layer dense_2420 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because its dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



...

 55%|███████████████████████████████████████████▌                                    | 109/200 [13:24<11:11,  7.38s/it]2021-02-23 00:48:56,624 - <ipython-input-27-65569484266e>[line:53] - INFO: At round 109, test loss is: 0.2295, metrics result is [0.9328421], train loss is 0.1531631052494049


 68%|██████████████████████████████████████████████████████▊                         | 137/200 [16:50<07:38,  7.28s/it]2021-02-23 00:52:22,360 - <ipython-input-27-65569484266e>[line:53] - INFO: At round 137, test loss is: 0.2075, metrics result is [0.93693054], train loss is 0.11479360610246658


 82%|██████████████████████████████████████████████████████████████████              | 165/200 [20:15<04:18,  7.37s/it]2021-02-23 00:55:47,174 - <ipython-input-27-65569484266e>[line:53] - INFO: At round 165, test loss is: 0.1966, metrics result is [0.93953085], train loss is 0.08937752991914749


 96%|█████████████████████████████████████████████████████████████████████████████▏  | 193/200 [23:42<00:51,  7.38s/it]2021-02-23 00:59:14,481 - <ipython-input-27-65569484266e>[line:53] - INFO: At round 193, test loss is: 0.1916, metrics result is [0.94030225], train loss is 0.07137907296419144


100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [24:34<00:00,  7.37s/it]
